# Train and visualize a model in Tensorflow - Part 2: The Basics of Tensorflow

This notebook will explain the basics to create a linear model with Tensorflow. This part uses the 20 newsgroup dataset obtained in the previous part of the tutorial. Remember the dataset, comprised of documents, was converted to a matrix of document embeddings and splitted into train and test datasets.

In [1]:
import numpy as np
import tensorflow as tf

## Data management

Before creating the model, we need to specify what the input and output is going to be. For that we use the document matrix obtained in the previous part as input to the the classifier.

However, most optimization algorithms similar to Stochastic Gradient Descent need the data in small portions for optimization purposes. On top of that, the training cycle goes through the entire dataset several times (epochs) before converging to a good solution.

Fortunately, Tensorflow has the solution to iterate over datasets several times in small batches. These function are called input functions, and they can take a numpy array or a pandas dataframe. It's worth noticing that, during the past updates, Tensorflow has been including more functions to transform the input data in batches handling enconding of categorical features, embeddings, etc, althoug we wont use those function here.

We load our dataset and create the input function to handle it with the following code:

In [2]:
# Load the dataset into a numpy keyed structure
newsgroups = np.load('./resources/newsgroup.npz')

# Define the batch size and the number of labels
batch_size = 100
num_classes = newsgroups['labels'].shape[0]

def dataset_input_fn(dataset):
    """
    Creates an input function using the `numpy_input_fn` method from
    tensorflow, based on the dataset we want to use.
    
    Args:
        dataset: String that represents the dataset (should be `train` or `test`)
    
    Returns:
        An `numpy_input_fn` function to feed to an estimator
    """
    assert dataset in ('train', 'test'), "The selected dataset should be `train` or `test`"
    
    return tf.estimator.inputs.numpy_input_fn(
        # A dictionary of numpy arrays that match each array with the corresponding column in the model.
        # For this case we only have "one" colum which represents all the dimensions in the embeddings.
        x={'input_data': newsgroups['%s_data' % dataset]},
        # The target array
        y=newsgroups['%s_target' % dataset],
        # The batch size to iterate the data in small fractions
        batch_size=batch_size,
        # If the dataset is `test` only run once
        num_epochs=1 if dataset == 'test' else None,
        # Only shuffle the dataset for the `train` data
        shuffle=dataset == 'train'
    )

## Defining the model

The classifier to train is a `tf.estimator.LinearClassifier` which is basically a wrapper in Tensorflow for a Logistic Regression classifier. 

The object instantiation takes as input an iterator (i.e. `feature_columns`) that match the dictionary fed to the input function. As the input function only takes one column with a number of dimensions equal to the number of dimensions in the embeddings, there is only one feature column of that number of dimensions.

In [3]:
embedding_size = newsgroups['train_data'].shape[1]

feature_columns = [tf.feature_column.numeric_column('input_data', shape=(embedding_size,))]

linear_classifier = tf.estimator.LinearClassifier(feature_columns=feature_columns, n_classes=num_classes)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_task_id': 0, '_num_worker_replicas': 1, '_task_type': 'worker', '_master': '', '_model_dir': '/tmp/tmprrqmkyhl', '_session_config': None, '_is_chief': True, '_service': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f30fef30080>}


## Training cicle

Now that we have the function that build the model, we can create the training cycle.

In [4]:
linear_classifier.train(input_fn=dataset_input_fn("train"), steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmprrqmkyhl/model.ckpt.
INFO:tensorflow:step = 1, loss = 299.573
INFO:tensorflow:global_step/sec: 511.301
INFO:tensorflow:step = 101, loss = 87.8829 (0.197 sec)
INFO:tensorflow:global_step/sec: 621.427
INFO:tensorflow:step = 201, loss = 104.938 (0.161 sec)
INFO:tensorflow:global_step/sec: 678.725
INFO:tensorflow:step = 301, loss = 91.5199 (0.147 sec)
INFO:tensorflow:global_step/sec: 649.765
INFO:tensorflow:step = 401, loss = 67.8664 (0.154 sec)
INFO:tensorflow:global_step/sec: 673.782
INFO:tensorflow:step = 501, loss = 53.9578 (0.148 sec)
INFO:tensorflow:global_step/sec: 650.838
INFO:tensorflow:step = 601, loss = 68.7622 (0.154 sec)
INFO:tensorflow:global_step/sec: 698.783
INFO:tensorflow:step = 701, loss = 61.8966 (0.143 sec)
INFO:tensorflow:global_step/sec: 718.016
INFO:tensorflow:step = 801, loss = 70.681 (0.139 sec)
INFO:tensorflow:global_step/sec: 691.578
INFO:tensorflow:step = 901, loss

## Evaluation

As seen before, it is also quite easy to get the evaluation metrics defined in the model after traning:

In [5]:
# Evaluate the model and print results
eval_results = linear_classifier.evaluate(input_fn=dataset_input_fn("test"))
print(eval_results)

INFO:tensorflow:Starting evaluation at 2017-11-15-13:53:04
INFO:tensorflow:Restoring parameters from /tmp/tmprrqmkyhl/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-11-15-13:53:05
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.759426, average_loss = 0.75266, global_step = 2000, loss = 74.5926
{'global_step': 2000, 'accuracy': 0.75942647, 'average_loss': 0.75266039, 'loss': 74.592606}
